In [37]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_table('diamonds.txt')

Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень кальция в этой популяции отличается от 9.5?

Посчитайте достигаемый уровень значимости. Поскольку известны только среднее и дисперсия, а не сама выборка, нельзя использовать стандартные функции критериев — нужно реализовать формулу достигаемого уровня значимости самостоятельно.

Округлите ответ до четырёх знаков после десятичной точки.

Реализуем формулу: $Z(X^n) = \frac{\overline{X}-\mu_0}{\frac{\sigma}{\sqrt{n}}}$

In [14]:
def z_conf(mu, sigma, n, x_mean):
    return (x_mean - mu)/(sigma / np.sqrt(n)) 

In [18]:
round(2*(1 - stats.norm.cdf(z_conf(n = 160, sigma =0.4, x_mean = 9.57 , mu = 9.5))),4)

0.0269

Имеются данные о стоимости и размерах 53940 бриллиантов:

Отделите $25\%$ случайных наблюдений в тестовую выборку с помощью функции $\textbf{sklearn.cross_validation.train_test_split}$ (зафиксируйте $\textbf{random state = 1}$). На обучающей выборке настройте две регрессионные модели:

1. линейную регрессию с помощью $\textbf{LinearRegression}$ без параметров
2. случайный лес с помощью RandomForestRegressor с $\textbf{random_state=1}$.

Какая из моделей лучше предсказывает цену бриллиантов? Сделайте предсказания на тестовой выборке, посчитайте модули отклонений предсказаний от истинных цен. Проверьте гипотезу об одинаковом среднем качестве предсказаний, вычислите достигаемый уровень значимости. Отвергается ли гипотеза об одинаковом качестве моделей против двусторонней альтернативы на уровне значимости $\alpha=0.05$?

In [27]:
X = df.drop(['price'], axis=1)
y = df['price']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)

In [39]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)
explained_variance_score(y_test, linreg.predict(X_test))

0.8602990074285857

In [41]:
rand_forest = RandomForestRegressor(random_state=1)
rand_forest.fit(X_train, y_train)
explained_variance_score(y_test, rand_forest.predict(X_test))

/home/danone/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8675733837867867

In [50]:
predict_linreg = linreg.predict(X_test) - y_test

In [52]:
predict_linreg.describe()

count    10788.000000
mean        17.572670
std       1473.413312
min     -12464.218640
25%       -344.468409
50%         63.846223
75%        647.219019
max      18348.588793
Name: price, dtype: float64

In [53]:
predict_rand_forest = rand_forest.predict(X_test) - y_test

In [54]:
predict_rand_forest.describe()

count    10788.000000
mean        46.189232
std       1434.539412
min     -12662.600000
25%       -252.825000
50%         33.431250
75%        406.230000
max       8047.700000
Name: price, dtype: float64

In [55]:
stats.ttest_rel(np.abs(predict_linreg), np.abs(predict_rand_forest))

Ttest_relResult(statistic=9.7923616963699, pvalue=1.5074001779559363e-22)

В предыдущей задаче посчитайте 95% доверительный интервал для разности средних абсолютных ошибок предсказаний регрессии и случайного леса. Чему равна его ближайшая к нулю граница? Округлите до десятков (поскольку случайный лес может давать немного разные предсказания в зависимости от версий библиотек, мы просим вас так сильно округлить, чтобы полученное значение наверняка совпало с нашим).



In [66]:
DescrStatsW(np.abs(predict_linreg)-np.abs(predict_rand_forest)).tconfint_mean()

(59.92901030520697, 89.92625661775124)